## Wanted

In [ ]:
from selenium import webdriver
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions


chrome_options = ChromeOptions()
chrome_options.add_experimental_option("detach", True) # 실행할 때마다 새 창 안 열리게 설정
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
driver.get('https://www.wanted.co.kr/wdlist?country=kr&job_sort=company.response_rate_order&years=-1&locations=all')
act = ActionChains(driver)  # 드라이버에 동작을 실행시키는 명령어를 act로 지정

#### 로그인

In [ ]:
element_login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#__next > div.NavBar_className__y2qGg > div > nav > aside > ul > li:nth-child(2) > button')))
act.move_to_element(element_login).click().perform()

element_id = driver.find_element(By.CSS_SELECTOR, "#__next > div > div > div > div > form > div:nth-child(3) > input")
element_submit = driver.find_element(By.CSS_SELECTOR, "#__next > div > div > div > div > form > button.css-c61xw1")
act.move_to_element(element_id)\
    .click_and_hold()\
    .send_keys("ija06598@gmail.com")\
    .move_to_element(element_submit)\
    .click()\
    .perform()

element_password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#__next > div > div > div > div.css-yc37i7 > form > div.css-1ctuhaz > input")))
element_submit = driver.find_element(By.CSS_SELECTOR, "#__next > div > div > div > div.css-yc37i7 > form > button.css-c61xw1")
act.move_to_element(element_password)\
    .click()\
    .send_keys("sowlq!0516")\
    .move_to_element(element_submit)\
    .click()\
    .perform()

#### 카테고리 선택

In [ ]:
element_dev = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/button')
act.move_to_element(element_dev).click().perform()

element_웹개발자 = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[2]')
element_프론트 = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[5]')
element_자바 = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[6]')
element_파이썬 = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[8]')
element_Node = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[10]')
element_submit = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[2]/button/span[2]')
act.move_to_element(element_웹개발자).click()\
    .move_to_element(element_프론트).click()\
    .move_to_element(element_자바).click()\
    .move_to_element(element_파이썬).click()\
    .move_to_element(element_Node).click()\
    .move_to_element(element_submit).click()\
    .perform()

element_career = driver.find_element(By.XPATH, "//*[@id='__next']/div[3]/div/div/div[1]/div[1]/div/div/div[1]/button")
act.move_to_element(element_career).click().perform()

draggable = driver.find_element(By.XPATH, "//*[@id='__next']/div[3]/div/div/div[1]/div/div/div/div[1]/section/div/div/div/div[5]/div")
start = draggable.location
finish = driver.find_element(By.XPATH, "//*[@id='__next']/div[3]/div/div/div[1]/div/div/div/div[1]/section/div/div/div/div[4]/div").location
element_apply = driver.find_element(By.XPATH, "//*[@id='__next']/div[3]/div/div/div[1]/div/div/div/div[1]/section/footer/div/button[2]/span[2]")
act.drag_and_drop_by_offset(draggable, finish['x'] - start['x'], finish['y'] - start['y'])\
    .move_to_element(element_apply).click()\
    .perform()

#### 크롤링

In [ ]:
# 데이터 다 렌더링될 때까지 스크롤
def scroll(driver, act):
    from time import sleep
    before = driver.execute_script('return window.scrollY')
    while True:
        act.send_keys(Keys.END).perform()
        sleep(3)
        companies = driver.find_elements(By.CLASS_NAME, "body")
        after = driver.execute_script('return window.scrollY')
        if before == after: break
        before = after

# 요소별 데이터 추출
def crawl(name):
    scroll(driver, act)
    data = driver.find_elements(By.CLASS_NAME, "body")
    data = list(map(lambda i : i.text.split("\n"), data))

    for i in data:
        if len(i) == 4:
            i.insert(2, " ")
        i.insert(4, name)
    data = list(map(lambda item: [item[0], item[1], item[4]], data))

    return data

# 회사별 기술스택 추출
def find_skill(length):

    element_post = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[4]/ul/li[' + str(length + 1) + ']')
            ))
    act.move_to_element(element_post).click().perform()
    sleep(2)

    skill = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/div[1]/div/div[2]/section[1]/p[6]/div')
    if skill == []:
        skill = ['null']
    else:
        skill = list(map(lambda i: i.text.split('\n'), skill))
    driver.back()
    print(i)

    return skill

# csv로 데이터 저장
def save(data, title, row):
    import csv
    # Specify the path and filename for the CSV file
    csv_file = title

    # Open the CSV file in write mode
    with open(csv_file, 'w', newline='', encoding='euc-kr') as file:
        writer = csv.writer(file)

        # Write the header row
        writer.writerow(row)

        # Write the data rows
        writer.writerows(data)

    print('Data saved to', csv_file)

In [ ]:
element_post = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[4]/ul/li[24]')

In [ ]:
from time import sleep

# companies, skills 크롤링

scroll(driver, act)

companies = driver.find_elements(By.CLASS_NAME, "body")
companies = list(map(lambda i : i.text.split("\n"), companies))

for i in companies:
    if len(i) == 4:
        i.insert(2, " ")
        
len(companies)
skills = []
for i in range(1, len(companies)):
    skills += find_skill(i)
    

In [ ]:
# companies + skills 후 저장

filtered_companies = list(map(lambda i : [i[1], i[0], i[3]], companies))

for i in range(len(filtered_companies)):
    filtered_companies[i].insert(2, skills[i])

save(data=filtered_companies, title='data_wanted.csv', row=['Company', 'Position', 'Skill', 'Location'])

## Programmers

In [ ]:
chrome_options = ChromeOptions()
chrome_options.add_experimental_option("detach", True) # 실행할 때마다 새 창 안 열리게 설정
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
driver.get('https://career.programmers.co.kr/job?page=1&min_career=0&order=recent&job_category_ids=4&job_category_ids=1&job_category_ids=25')
act = ActionChains(driver)  # 드라이버에 동작을 실행시키는 명령어를 act로 지정

In [ ]:
from time import sleep
import csv

# 페이지별 데이터 추출
def crawl():

    element_next = driver.find_element(By.XPATH, '//*[@id="tab_position"]/div[3]/ul/li[9]/span')
    act.move_to_element(element_next).click().perform()
    sleep(2)
    
    data = driver.find_elements(By.CLASS_NAME, "item-body")
    data = list(map(lambda i : i.text.split('\n'), data))

    return data

# 회사별 기술스택 추출
def find_skill(length):
    skills = []

    for i in range(1, length + 1):
        element_post = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="list-positions-wrapper"]/ul/li[' + str(i) + ']')
            ))
        act.move_to_element(element_post).click().perform()
        
        sleep(2)

        skill = driver.find_elements(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/div[2]/section/ul')
        if skill == []:
            skills += ['null']
        else:
            skills += list(map(lambda i: i.text.split('\n'), skill))
            
        driver.back()

    return skills

# csv로 데이터 저장
def save(data, title, row):
    # Specify the path and filename for the CSV file
    csv_file = title

    # Open the CSV file in write mode
    with open(csv_file, 'w', newline='', encoding='euc-kr') as file:
        writer = csv.writer(file)

        # Write the header row
        writer.writerow(row)

        # Write the data rows
        writer.writerows(data)

    print('Data saved to', csv_file)

In [ ]:
# companies, skills 크롤링

# 1 페이지 크롤링
companies = driver.find_elements(By.CLASS_NAME, "item-body")
companies = list(map(lambda i : i.text.split('\n'), companies))

skills = []
for i in range(1, 21):
    element_post = driver.find_element(By.XPATH, '//*[@id="list-positions-wrapper"]/ul/li[' + str(i) + ']')
    act.move_to_element(element_post).click().perform()
    sleep(2)
    skill = driver.find_elements(By.XPATH, '//*[@id="career-app-legacy"]/div/div[1]/div[1]/div[2]/section/ul')
    if skill == []:
        skills += ['null']
    else:
        skills += list(map(lambda i: i.text.split('\n'), skill))
    driver.back()
    sleep(2)
len(skills)

# 나머지 페이지 크롤링

for i in range(6):
    print(i + 2)
    company = crawl()
    companies += company
    skills += find_skill(len(company))
len(companies)

In [ ]:
# companies + skills 후 저장

filtered_companies = []
for company in companies:
    if len(company) >= 5:  # Check if the inner list has at least 5 elements
        filtered_companies.append([company[1], company[4], company[3]])

for i in range(len(filtered_companies)):
    filtered_companies[i].insert(2, skills[i])

save(data=filtered_companies, title='data_programmers.csv', row=['Company', 'Position', 'Skill', 'Location'])